In [6]:
print(base.__file__)

/Users/alaink/.local/lib/python3.9/site-packages/StructuralGT-1.0.1b2-py3.9.egg/StructuralGT/base.py


In [1]:
#Draft
import numpy as np
import igraph as ig
import os
import cv2 as cv
from StructuralGT import process_image
#import error, base, process_image
import json
import base
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
import functools
import gsd.hoomd
from skimage.morphology import skeletonize_3d, disk
import pandas as pd

class Network():
    """Generic SGT graph class: a specialised case of the igraph Graph object with 
    additional attributes defining geometric features, associated images,
    dimensionality etc.
    
    Initialised from directory containing raw image data
    self._2d determined from the number of images with identical dimensions (suggesting a stack when > 1)
    
    Image shrinking/cropping is carried out at the gsd stage in analysis.
    I.e. full images are binarized but cropping their graphs may come after
    """
    def __init__(self, directory, child_dir='/Binarized'):
        if not isinstance(directory, str):
            raise TypeError
            
        self.dir = directory
        self.child_dir = child_dir
        self.stack_dir = self.dir + self.child_dir
        
        shape = []
        for name in sorted(os.listdir(self.dir)):
            if not base.Q_img(name):
                continue
            shape.append(cv.imread(self.dir+'/'+name,cv.IMREAD_GRAYSCALE).shape)
        
        if len(set(shape)) == len(shape):
            self._2d = True
            self.dim = 2
        else:
            self._2d = False
            self.dim = 3
        
    def binarize(self, options_dict=None):
        """Binarizes stack of experimental images using a set of image processing parameters in options_dict.
        Note this enforces that all images have the same shape as the first image encountered by the for loop.
        (i.e. the first alphanumeric titled image file)
        """
        
        if options_dict is None:
            options = self.dir + '/img_options.json'
            with open(options) as f:
                options_dict = json.load(f)

        if not os.path.isdir(self.dir + self.child_dir):
            os.mkdir(self.dir + self.child_dir)

        i=0
        for name in sorted(os.listdir(self.dir)):
            if not base.Q_img(name):
                continue
            else:
                img_exp = cv.imread(self.dir+'/'+name,cv.IMREAD_GRAYSCALE)
                if i == 0: shape = img_exp.shape
                elif img_exp.shape != shape: continue
                _, img_bin, _ = process_image.binarize(img_exp, options_dict)
                plt.imsave(self.dir+self.child_dir+'/slice'+str(i)+'.tiff', img_bin, cmap=cm.gray)
                i+=1
        
    def stack_to_gsd(self, name='skel.gsd', crop=None, skeleton=True, rotate=None, debubble=None):
        """Writes a .gsd file from the object's directory.
        The name of the written .gsd is set as an attribute so it may be easily matched with its Graph object 
        Running this also sets the positions, shape attributes
        
        For 2D graphs, the first element of the 3D position list is all 0s. So the gsd y corresponds to the graph
        x and the gsd z corresponds to the graph y.
        """
        start = time.time()
        
        if name[0] == '/':
            self.gsd_name = name
        else:
            self.gsd_name = self.stack_dir + '/' + name
        self.gsd_dir = os.path.split(self.gsd_name)[0]
        img_bin=[]

        #Initilise i such that it starts at the lowest number belonging to the images in the stack_dir
        #First require boolean mask to filter out non image files
        olist = np.asarray(sorted(os.listdir(self.stack_dir)))
        mask = list(base.Q_img(olist[i]) for i in range(len(olist)))
        if len(mask) == 0:
            raise error.ImageDirectoryError(self.stack_dir)
        fname = sorted(olist[mask])[0] #First name
        i = int(os.path.splitext(fname)[0][5:]) #Strip file type and 'slice' then convert to int

        #Generate 3d (or 2d) array from stack
        for fname in sorted(os.listdir(self.stack_dir)):
            if base.Q_img(fname):
                img_slice = cv.imread(self.stack_dir+'/slice'+str(i)+'.tiff',cv.IMREAD_GRAYSCALE)
                if rotate is not None:
                    image_center = tuple(np.array(img_slice.shape[1::-1]) / 2)
                    rot_mat = cv.getRotationMatrix2D(image_center, rotate, 1.0)
                    img_slice = cv.warpAffine(img_slice, rot_mat, img_slice.shape[1::-1], flags=cv.INTER_LINEAR)
                img_bin.append(img_slice)
                i=i+1
            else:
                continue

        #For 2D images, img_bin_3d.shape[0] == 1
        img_bin = np.asarray(img_bin)
        self.img_bin_3d = img_bin
        self.img_bin = img_bin

        #Note that numpy array slicing operations are carried out in reverse order!
        #(...hence crop 2 and 3 before 0 and 1)
        if crop and self._2d:
            self.img_bin = self.img_bin[:, crop[2]:crop[3], crop[0]:crop[1]]
            #img_bin = img_bin[crop[0]:crop[1], crop[2]:crop[3]]
        elif crop:
            #TODO figure tf this bit out
            self.img_bin = self.img_bin[crop[0]:crop[1], crop[2]:crop[3], crop[4]:crop[5]]

        assert self.img_bin_3d.shape[1] > 1
        assert self.img_bin_3d.shape[2] > 1
        
        self.img_bin_3d = self.img_bin            #Always 3d, even for 2d images
        self.img_bin = np.squeeze(self.img_bin)   #3d for 3d images, 2d otherwise

        assert self.img_bin_3d.shape[1] == self.img_bin.shape[0]
        assert self.img_bin_3d.shape[2] == self.img_bin.shape[1]
        
        if skeleton:
            self.skeleton = skeletonize_3d(np.asarray(self.img_bin))
            self.skeleton_3d = skeletonize_3d(np.asarray(self.img_bin_3d))
            #self.skeleton_3d = np.swapaxes(self.skeleton_3d, 1, 2)
        else:
            self.img_bin = np.asarray(self.img_bin)

        positions = np.asarray(np.where(np.asarray(self.skeleton_3d) == 255)).T
        self.shape = np.asarray(list(max(positions.T[i])+1 for i in (0,1,2)[0:self.dim]))
        self.positions = positions

        print(positions)
        print(self.img_bin.shape)

        with gsd.hoomd.open(name=self.gsd_name, mode='wb') as f:
            s = gsd.hoomd.Snapshot()
            s.particles.N = len(positions)
            s.particles.position = base.shift(positions)
            s.particles.types = ['A']
            s.particles.typeid = ['0']*s.particles.N
            f.append(s)

        end = time.time()
        print('Ran stack_to_gsd() in ', end-start, 'for gsd with ', len(positions), 'particles')

        if debubble is not None: self = base.debubble(self, debubble)

        assert self.img_bin.shape == self.skeleton.shape
        assert self.img_bin_3d.shape == self.skeleton_3d.shape

        
    def stack_to_circular_gsd(self, radius, name='circle.gsd', rotate=None, debubble=None, skeleton=True):
        """Writes a cicular .gsd file from the object's directory.
        Currently only capable of 2D graphs
        Unlike stack_to_gsd, the axis of rotation is not the centre of the image, but the point (radius,radius)
        The name of the written .gsd is set as an attribute so it may be easily matched with its Graph object 
        Running this also sets the positions, shape attributes.
        
        Note the rotation implementation is very different to self.stack_to_gsd():
        A rotating circular graph will never lose/gain nodes so no need to recalculate weights
        Instead
            Generate the graph at theta=0.
            Set all attributes.
            Apply rotation matrix to positional attributes
        """
        start = time.time()
        if name[0] == '/':
            self.gsd_name = name
        else:
            self.gsd_name = self.stack_dir + '/' + name
        self.gsd_dir = os.path.split(self.gsd_name)[0]
        img_bin=[]
        
        #Initilise i such that it starts at the lowest number belonging to the images in the stack_dir
        #First require boolean mask to filter out non image files
        olist = np.asarray(sorted(os.listdir(self.stack_dir)))
        mask = list(base.Q_img(olist[i]) for i in range(len(olist)))
        if len(mask) == 0:
            raise error.ImageDirectoryError(self.stack_dir)
        fname = sorted(olist[mask])[0] #First name
        i = int(os.path.splitext(fname)[0][5:]) #Strip file type and 'slice' then convert to int
        
        img_slice = cv.imread(self.stack_dir+'/slice'+str(i)+'.tiff',cv.IMREAD_GRAYSCALE)
        
        #Read the image
        for fname in sorted(os.listdir(self.stack_dir)):
            if base.Q_img(fname):
                img_slice = cv.imread(self.stack_dir+'/slice'+str(i)+'.tiff',cv.IMREAD_GRAYSCALE)
                img_bin.append(img_slice)
                i=i+1
            else:
                continue
                
        #For 2D images, img_bin_3d.shape[0] == 1
        img_bin = np.asarray(img_bin)
        
        self.img_bin_3d = img_bin            #Always 3d, even for 2d images
        self.img_bin = np.squeeze(img_bin)   #3d for 3d images, 2d otherwise
        
        #Note that numpy array slicing operations are carried out in reverse order!
        #(...hence crop 2 and 3 before 0 and 1)
        assert self._2d


        canvas = np.ones(self.img_bin.shape)
        disk_pos = np.asarray(np.where(disk(radius)!=0)).T
        canvas[disk_pos[0], disk_pos[1]] = 0
        self.img_bin = np.ma.MaskedArray(self.img_bin, mask=canvas)
        self.img_bin = np.ma.filled(self.img_bin, fill_value=0)
        
        canvas = np.ones(self.img_bin_3d.shape)
        disk_pos = np.asarray(np.where(disk(radius)!=0)).T
        disk_pos = np.array([np.zeros(len(disk_pos)), disk_pos.T[0], disk_pos.T[1]], dtype=int)
        canvas[disk_pos[0], disk_pos[1], disk_pos[2]] = 0
        self.img_bin_3d = np.ma.MaskedArray(self.img_bin_3d, mask=canvas)
        self.img_bin_3d = np.ma.filled(self.img_bin_3d, fill_value=0)
        self.img_bin = self.img_bin_3d[0]
        
        assert self.img_bin_3d.shape[1] > 1
        assert self.img_bin_3d.shape[2] > 1
        
        if skeleton:
            self.skeleton = skeletonize_3d(np.asarray(self.img_bin))
            self.skeleton_3d = skeletonize_3d(np.asarray(self.img_bin_3d))
        else:
            self.img_bin = np.asarray(self.img_bin)
        
        positions = np.asarray(np.where(np.asarray(self.skeleton_3d) == 255)).T
        self.shape = np.asarray(list(max(positions.T[i])+1 for i in (0,1,2)[0:self.dim]))
        self.positions = positions
        
        with gsd.hoomd.open(name=self.gsd_name, mode='wb') as f:
            s = gsd.hoomd.Snapshot()
            s.particles.N = len(positions)
            s.particles.position = base.shift(positions)
            s.particles.types = ['A']
            s.particles.typeid = ['0']*s.particles.N
            f.append(s)
        
        end = time.time()
        print('Ran stack_to_gsd() in ', end-start, 'for gsd with ', len(positions), 'particles')
        
        if debubble is not None: self = base.debubble(self, debubble)
            
        assert self.img_bin.shape == self.skeleton.shape
        assert self.img_bin_3d.shape == self.skeleton_3d.shape    
        
        """Set rot matrix attribute for later"""
        if rotate is not None:
            from scipy.spatial.transform import Rotation as R
            r = R.from_rotvec(rotate/180*np.pi * np.array([0, 0, 1]))
            self.rotate = r.as_matrix()
            
        
    def G_u(self):
        """Sets unweighted igraph object as an attribute
        """
        G =  base.gsd_to_G(self.gsd_name, _2d = self._2d)
        self.Gr = G
        self.shape = list(max(list(self.Gr.vs[i]['o'][j] for i in range(self.Gr.vcount()))) for j in (0,1,2)[0:self.dim])
        
    def weighted_Laplacian(self, weights='weight'):

        L=np.asarray(self.Gr.laplacian(weights=weights))
        self.L = L

class ResistiveNetwork(Network):
    """Child of generic SGT Network class.
    Equipped with methods for analysing resistive flow networks
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def potential_distribution(self, plane, boundary1, boundary2, R_j=0, rho_dim=1, F_dim=1):
        """Solves for the potential distribution in a weighted network.
        Source and sink nodes are connected according to a penetration boundary condition.
        Sets the corresponding weighted Laplacian, potential and flow attributes.
        The 'plane' arguement defines the axis along which the boundary arguements refer to.
        R_j='infinity' enables the unusual case of all edges having the same unit resistance.
        
        NOTE: Critical that self.G_u() is called before every self.potential_distribution()
        TODO: Remove this requirement or add an error to warn
        """
        self.Gr = base.sub_G(self.Gr)
        print('post sub has ', self.Gr.vcount(), ' nodes')
        if R_j != 'infinity':
            print(self.Gr.vcount())
            self.Gr_connected = base.add_weights(self, weight_type='Conductance', R_j=R_j, rho_dim=rho_dim)
            print(self.Gr.vcount())
            weight_array = np.asarray(self.Gr_connected.es['Conductance']).astype(float)
            weight_array = weight_array[~np.isnan(weight_array)]
            self.edge_weights = weight_array
            weight_avg =np.mean(weight_array)
        else:
            self.Gr_connected = self.Gr
            self.Gr_connected.es['Conductance'] = np.ones(self.Gr_connected.ecount())
            weight_avg = 1

            
        """Apply rotation for circular gsds"""
        if self.rotate is not None:
            centre = np.asarray(self.shape)/2
            node_positions = np.asarray(list(self.Gr_connected.vs[i]['o'] for i in range(self.Gr_connected.vcount())))
            node_positions = base.oshift(node_positions, _shift=-centre)
            print(node_positions)
            node_positions = np.vstack((node_positions.T, np.zeros(len(node_positions)))).T
            node_positions = np.matmul(node_positions, self.rotate).T[0:2].T
            node_positions = base.shift(node_positions, _shift=centre)
            for i in range(self.Gr_connected.vcount()):
                print(node_positions[i],i)
                self.Gr_connected.vs[i]['o'] = node_positions[i]
                
            
            edge_positions_list = np.asarray(list(base.oshift(self.Gr_connected.es[i]['pts'], _shift=-centre) for i in range(self.Gr_connected.ecount())))
            for i, edge in enumerate(edge_positions_list):
                edge_position = np.vstack((edge.T, np.zeros(len(edge)))).T
                edge_position = np.matmul(edge_position, self.rotate).T[0:2].T
                self.Gr_connected.es[i]['pts'] = base.shift(edge_position, _shift=centre)
                
        #Add source and sink nodes:
        source_id = max(self.Gr_connected.vs).index + 1
        sink_id = source_id + 1
        self.Gr_connected.add_vertices(2)

        print('Graph has max ', self.shape)
        axes = np.array([0,1,2])[0:self.dim]
        indices = axes[axes!=plane]
        plane_centre1 = np.zeros(self.dim, dtype=int)
        delta = np.zeros(self.dim, dtype=int)
        delta[plane] = 10 #Arbitrary. Standardize?
        for i in indices: plane_centre1[i] = self.shape[i]/2
        plane_centre2 = np.copy(plane_centre1)
        plane_centre2[plane] = self.shape[plane]
        source_coord = plane_centre1 - delta 
        sink_coord = plane_centre2 + delta
        print('source coord is ', source_coord)
        print('sink coord is ', sink_coord)
        self.Gr_connected.vs[source_id]['o'] = source_coord
        self.Gr_connected.vs[sink_id]['o'] = sink_coord

    #Connect nodes on a given boundary to the external current nodes
        print('Before connecting external nodes, G has vcount ', self.Gr_connected.vcount())
        for node in self.Gr_connected.vs:
            if node['o'][plane] > boundary1[0] and node['o'][plane] < boundary1[1]:
                self.Gr_connected.add_edges([(node.index, source_id)])
                self.Gr_connected.es[self.Gr_connected.get_eid(node.index,source_id)]['Conductance'] = weight_avg
                self.Gr_connected.es[self.Gr_connected.get_eid(node.index,source_id)]['o'] = base.connector(source_coord,node['o'])
            if node['o'][plane] > boundary2[0] and node['o'][plane] < boundary2[1]:
                self.Gr_connected.add_edges([(node.index, sink_id)])
                self.Gr_connected.es[self.Gr_connected.get_eid(node.index,sink_id)]['Conductance'] = weight_avg 
                self.Gr_connected.es[self.Gr_connected.get_eid(node.index,sink_id)]['o'] = base.connector(sink_coord,node['o'])

    #Write skeleton connected to external node
        print(self.Gr_connected.is_connected(), ' connected')
        print('After connecting external nodes, G has vcount ', self.Gr_connected.vcount())
        connected_name = os.path.split(self.gsd_name)[0] + '/connected_' + os.path.split(self.gsd_name)[1] 
        #connected_name = self.stack_dir + '/connected_' + self.gsd_name 
        base.G_to_gsd(self.Gr_connected, connected_name)
        
        if R_j=='infinity': self.L = np.asarray(self.Gr.laplacian())
        else: self.weighted_Laplacian(weights='Conductance')
        F = np.zeros(sink_id+1)
        print(F.shape,'F')
        print(self.L.shape, 'L')
        F[source_id] = F_dim
        F[sink_id] = -F_dim
        np.save(self.stack_dir+'/L.npy',self.L)
        np.save(self.stack_dir+'/F.npy',F)
        P = np.matmul(np.linalg.pinv(self.L, hermitian=True),F)
        np.save(self.stack_dir+'/P.npy',P)

        self.P = P
        self.F = F


In [2]:
scale=1
img_options={"Thresh_method":1, "gamma": 2.34, "md_filter": 0, "g_blur": 1, "autolvl": 0,
             "fg_color":1, "laplacian": 0, "scharr": 1, "sobel":0 , "lowpass": 1, "asize": int((103*scale))*2+1,
             "bsize":int((43*scale))*2+1, "wsize": int((10*scale))*2+1, "thresh": 127}


"""
img_options={"Thresh_method":1, "gamma": 2.34, "md_filter": 0, "g_blur": 1, "autolvl": 0,
             "fg_color":1, "laplacian": 0, "scharr": 1, "sobel":1 , "lowpass": 1, "asize": 103,
             "bsize":43, "wsize": 10, "thresh": 127}
"""
f = ResistiveNetwork('TestData/Mxene/1')
#f.binarize(options_dict=img_options)
f.stack_to_circular_gsd(500, debubble=[disk(5)], rotate=10)
f.G_u()
f.potential_distribution(0, [0,20], [480,500])

Ran stack_to_gsd() in  0.747974157333374 for gsd with  10673 particles
Ran debubble in  1.2199816703796387 for an image with shape  (1, 1757, 2047)
gsd_to_G canvas has shape  (995, 993)
[[ 16 428]]
(0, 2, array([[ 16, 428],
       [ 17, 429],
       [ 18, 430],
       [ 18, 431],
       [ 18, 432],
       [ 18, 433],
       [ 18, 434],
       [ 18, 435],
       [ 18, 436],
       [ 18, 437],
       [ 18, 438],
       [ 19, 439],
       [ 20, 440],
       [ 20, 441],
       [ 21, 442],
       [ 21, 443],
       [ 22, 444],
       [ 23, 445],
       [ 24, 446],
       [ 25, 447],
       [ 26, 448],
       [ 27, 449],
       [ 27, 450],
       [ 28, 451],
       [ 29, 452],
       [ 30, 453],
       [ 31, 454],
       [ 32, 455],
       [ 33, 456],
       [ 34, 457],
       [ 34, 458],
       [ 35, 459],
       [ 35, 460],
       [ 36, 461],
       [ 36, 462],
       [ 37, 463],
       [ 38, 464],
       [ 39, 465],
       [ 40, 465],
       [ 41, 466],
       [ 42, 467],
       [ 43, 468

/Users/alaink/miniconda3/envs/v3_env/lib/python3.9/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [4]:
print(list(f.Gr_connected.vs[i]['o'] for i in range(30)))

[array([476.31866541, 716.40413046]), array([529.62206366, 650.14141292]), array([541.15194589, 634.90783764]), array([545.84044669, 661.49764698]), array([561.54028942, 652.63678144]), array([558.25539948, 530.34937613]), array([598.22813968, 515.17769064]), array([613.22858123, 536.90294676]), array([582.14796167, 343.35968263]), array([605.53338434, 470.22623459]), array([634.72070362, 532.0978791 ]), array([603.17622084, 341.68268641]), array([635.3011337 , 523.87212072]), array([646.71204483, 530.99890872]), array([637.8013107 , 457.42854228]), array([582.64548423,  98.55414252]), array([590.17424566, 112.45801793]), array([656.33660559, 470.40709549]), array([596.73022053,  40.22215984]), array([605.52919628,  84.36486055]), array([701.70686473, 382.18839234]), array([715.6083359 , 391.92230724]), array([ 661.0569819 , -147.80461424]), array([ 685.84841967, -133.8983346 ]), array([ 692.86104504, -157.47423512]), array([800.78730403, 218.49640552]), array([823.78037023, 250.997465

In [92]:
positions = np.asarray(list(f.Gr_connected.vs[i]['o'] for i in range(f.Gr_connected.vcount())))
for i in range(f.Gr_connected.ecount()):
    positions = np.append(positions,f.Gr_connected.es[i]['pts'], axis=0)
    
    

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 0 dimension(s)

In [100]:
print(list(f.Gr_connected.es[i]['pts'] for i in range(f.Gr_connected.ecount())))


[array([[397.73971604, 810.24262927],
       [398.89817197, 811.05378884],
       [400.0566279 , 811.86494842],
       [401.21508384, 812.67610799],
       [402.37353977, 813.48726757],
       [403.5319957 , 814.29842714],
       [404.69045163, 815.10958672],
       [405.67525938, 814.93593854],
       [406.83371531, 815.74709812],
       [407.99217124, 816.55825769],
       [409.15062717, 817.36941727],
       [410.13543493, 817.19576909],
       [411.29389086, 818.00692867],
       [412.45234679, 818.81808824],
       [413.61080272, 819.62924782],
       [414.76925865, 820.44040739],
       [415.92771458, 821.25156697],
       [417.08617051, 822.06272654],
       [418.24462644, 822.87388612],
       [419.40308237, 823.68504569],
       [420.5615383 , 824.49620527],
       [421.71999423, 825.30736484],
       [422.87845016, 826.11852442],
       [424.03690609, 826.92968399],
       [425.19536202, 827.74084357],
       [426.35381795, 828.55200314],
       [427.51227389, 829.36316272],


In [ ]:
"""Examples:
f = ResistiveNetwork('TestData/AgNWN_10um')
f.binarize()
f.stack_to_gsd(crop=[300,600,0,500], debubble=[disk(3)])
f.G_u()
f.potential_distribution(1, [0,20], [480,500])
base.Node_labelling(f, f.P, 'P', 'test1.gsd')

g = ResistiveNetwork('TestData/pores-hi-res-crop')
g.stack_to_gsd(crop=[0,100,0,100,0,100], debubble=[ball(3)])
g.G_u()
g.potential_distribution(0, [0,20], [80,100])
base.Node_labelling(g, g.P, 'P', 'test2.gsd')

s = StructuralNetwork('TestData/fibres-hi-res-crop')
s.stack_to_gsd(crop=[0,100,0,100,0,100])
s.G_u()
s.node_calc()
base.Node_labelling(s, s.Degree, 'Degree', 'test3.gsd')
"""

f = ResistiveNetwork('TestData/AgNWN_10um')
f.stack_to_circular_gsd(400, rotate=45, debubble=[disk(1)])
f.G_u()
f.potential_distribution(1, [0,50], [750,800])
Node_labelling(f, f.P, 'P', 'test1.gsd')

In [ ]:
#Rotation test
#Continuous rotation analysis
#Testing base.py implementation
#This implementation rotates the image itself

#######################################################
## USE THIS CELL FOR TESTING THE ABOVE DECLARATIONS ##
## USE A SEPARATE NOTEBOOK FOR TESTING network.py   ##
#######################################################

from PIL import Image
from skimage.morphology import binary_closing, disk, ball, skeletonize_3d
import network
g = network.ResistiveNetwork('TestData/AgNWN_10um')
g.binarize()

#thetas = np.linspace(0.001,350, 4)
thetas = (0,45)

#crop = 359

#Need to reassign weights from a temporary rotated binary image
#First write the temporary image
#Must calculate positions for the crop because this crop is origin cornered (and not origin centred, like the one above)
img_bin = cv.imread(g.stack_dir+'/slice0.tiff') #Original image
image_center = tuple(np.array(img_bin.shape[1::-1]) / 2)
short_length = img_bin.shape[img_bin.shape == max(img_bin.shape)]
long_length = max(img_bin.shape)
ISS = (short_length**2/2)**0.5
L1 = int((long_length - ISS)/2)
L3 = int(ISS+L1)
L2 = int((short_length - ISS)/2)
L4 = int(ISS+L2)
o_corn_crop = [L2,L1,L4,L3]
dims = L2-L1



R_j = 0
O_eff_df = []
Ax_df = []
Ay_df = []
theta_df = []
nodes = []
for theta in thetas:
    g.stack_to_gsd(name='Rotations/ObjectImplementation/rot_skel.gsd', crop=[L1,L3,L2,L4], rotate=theta, debubble=[disk(2)])
    g.G_u()
    measuring_graph = base.add_weights(g, weight_type='Resistance', R_j=R_j)
    g.potential_distribution(0, [0,20], [727-20,727], R_j=R_j, rho_dim=0.89, F_dim=1)
    base.Node_labelling(g, g.P, 'P', 'take2.gsd')
    print(g.Gr_connected.vcount())
    print(measuring_graph.vcount())
    
    #Ax, Ay = base.gyration_moments(g.Gr, sampling = 0.1)
    #L = g.L
    #Q = np.linalg.pinv(L)
    #O_eff = Q[-1,-1]+Q[-2,-2]-2*Q[-1,-2]
    
    nodes.append(g.Gr.vcount())
    df_cont = pd.DataFrame(columns=['Theta','O_eff','Ax','Ay'])
    df_cont['O_eff'] = O_eff_df
    df_cont['Ax'] = Ax_df
    df_cont['Ay'] = Ay_df
    df_cont['Theta'] = theta_df
    #df_cont.to_csv('AgNWN.csv')

    df_cont
    #print(theta, O_eff, Ax, Ay)
    
    #stop
    
    
    #O_eff_df.append(O_eff)
    #Ax_df.append(Ax)
    #Ay_df.append(Ay)
    #theta_df.append(theta)



In [ ]:
len(np.asarray(np.where(np.asarray(g.img_bin_3d)!=0)).T)

In [5]:
f.shape

[994, 992]

In [ ]:
g.img_bin_3d
img_bin = np.swapaxes(g.img_bin_3d, 0, 2)

crop = o_corn_crop
img_bin[crop[2]:crop[3], crop[0]:crop[1]].shape

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(thetas,nodes)
plt.show()

In [ ]:

img_bin=[]
img_bin.append(img_slice)
img_bin = np.asarray(img_bin)
img_bin = np.swapaxes(img_bin, 0, 2)
print(img_bin.shape)
img_bin = img_bin[crop[2]:crop[3],crop[0]:crop[1]]
print(img_bin.shape)

In [ ]:
def timer(_class,method):
    """Print the runtime of the decorated function"""
    @functools.wraps(method)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = _method(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {method.__name__!r} in {run_time:.4f} secs")
        print(_class)
        print(method)
        return value
    return wrapper_timer